In [ ]:
!pip3.10 install numpy pandas cplex networkx pyscipopt psutil py-cpuinfo tqdm matplotlib

# Import

In [1]:
import numpy as np
import pandas as pd
import cplex
import itertools
import networkx as nx
import pyscipopt as scip
import warnings
import os
import psutil
import cpuinfo 
import platform
from math import log, floor

# Dataset

In [4]:
datasets = [
    {"name": "Mobile operators", "path": "Mobile"},
    {"name": "Perfume (first 1000)", "path": "Perfume"},
    {"name": "Bibsonomy (last 1000)", "path": "Bibsonomy"},
    {"name": "IMDB", "path": "IMDB"}
]

def ds_reader(dataset):
    X=set()
    Y=set()
    Z=set()
    with open(f"Dataset/{dataset['path']}/12.txt") as f:
        for a in f:
            X.add(a.split()[0])
            Y.add(a.split()[1])
    with open(f"Dataset/{dataset['path']}/12.txt", 'rb') as f:
        G_XY=nx.read_edgelist(f)   
                
    with open(f"Dataset/{dataset['path']}/23.txt") as f:
        for a in f:
            Y.add(a.split()[0])
            Z.add(a.split()[1])

    with open(f"Dataset/{dataset['path']}/23.txt", 'rb') as f:
        G_YZ=nx.read_edgelist(f)
        
        
    with open(f"Dataset/{dataset['path']}/31.txt", 'rb') as f:
        G_ZX=nx.read_edgelist(f)        
        
    with open(f"Dataset/{dataset['path']}/31.txt") as f:
        for a in f:
            Z.add(a.split()[0])
            X.add(a.split()[1])
    X=list(X)
    Y=list(Y)
    Z=list(Z)
    return G_XY,G_YZ,G_ZX,X,Y,Z


def ds_load(dataset):
    match dataset['name']:
        case "Mobile operators":
            result = ds_reader(dataset)            
            return result
        case "Perfume (first 1000)":
            result = ds_reader(dataset)
            return result
        case "IMDB":
            result = ds_reader(dataset)
            return result           
        case "Bibsonomy (last 1000)":
            result = ds_reader(dataset)
            return result       
        case _:
            return "Unknown" 

# Solver

In [5]:

from tqdm import tqdm

def hw_info():
    # Определение операционной системы
    system = platform.system()

    if system == "Windows":
        os_type = "Windows"
    elif system == "Darwin":
        os_type = "MacOS"
    elif system == "Linux":
        os_type = "Linux"
    else:
        os_type = "Другая ОС"

    # Информация о процессоре
    cpu_info = cpuinfo.get_cpu_info()
    cpu_brand = cpu_info['brand_raw']
    cpu_count_logical = psutil.cpu_count(logical=True)
    cpu_freq = psutil.cpu_freq()

    # Информация о памяти
    virtual_memory = psutil.virtual_memory()

    info = f'{os_type} | {cpu_count_logical} x {cpu_brand} {cpu_freq.current} MHz | {virtual_memory.total / (1024 ** 3):.2f} GB'

    return info

df_results = pd.DataFrame()

def solve(dataset, model_name, gamma, seed_value=42):
    # Инициализация пустого DataFrame
    global df_results, timeout

    # Создание модели SCIP
    model = scip.Model()
    model.hideOutput()
    file_path = f'Model/{model_name}_g{gamma:.1f}_{dataset["path"]}.lp'
    model.readProblem(file_path)

    # Устанавливаем фиксированный сид для всех случайных процессов
    model.setParam('randomization/randomseedshift', seed_value)
    model.setParam('randomization/permutationseed', seed_value)
    model.setParam('randomization/permuteconss', True)
    model.setParam('randomization/permutevars', False)
    model.setParam('randomization/lpseed', seed_value)
    
    model.setParam("parallel/maxnthreads", 20)  # Установить количество потоков
    model.setParam("limits/time", timeout) # Установка временного лимита на оптимизацию 

    # Оптимизация модели
    model.optimize()

    # Проверка статуса решения
    if model.getStatus() == "optimal":
        best_sol = model.getBestSol()
        best_obj = model.getObjVal()
        print(f"Best objective value: {best_obj}")

        sold=eval(str(best_sol))
        x = [(key,sold[key]) for key in sold.keys() if key[0]=="x" and sold[key]>0.9]
        y = [(key,sold[key]) for key in sold.keys() if key[0]=="y" and key[1]!="(" and sold[key]>0.9]
        z = [(key,sold[key]) for key in sold.keys() if key[0]=="z" and key[1]!="(" and sold[key]>0.9]
        xfull = [(key,sold[key]) for key in sold.keys() if key[0]=="x"]
        yfull = [(key,sold[key]) for key in sold.keys() if key[0]=="y" and key[1]!="("]
        zfull = [(key,sold[key]) for key in sold.keys() if key[0]=="z" and key[1]!="("]

        xl, yl, zl = len(x), len(y), len(z) 

        solving_time = model.getSolvingTime() # Затраченное время
        num_solutions = model.getNSols() # Получение числа найденных решений
             
        print (f'({xl}, {yl}, {zl})')
        print(f"Time: {solving_time}")
        print(f"Solutions: {num_solutions}") 

         # Сохранение данных
        save_results_to_df(dataset, model_name, gamma, best_obj, xl, yl, zl, solving_time, num_solutions, x, y, z)

    else:
        print(f"No solution in time: {timeout}'s")
        save_results_to_df(dataset, model_name, gamma, "-", "-", "-", "-", f"{timeout}", f"No solution in time", "-", "-", "-")


# Функция для обновления DataFrame результатами
def save_results_to_df(dataset, model_name, gamma, best_obj, count_x, count_y, count_z, time, solutions, x, y, z):
    global df_results
    data = pd.DataFrame({
        'Dataset':  [dataset["name"]],
        'Model':  [model_name],
        'Gamma': [gamma],
        'First solution (X,Y,Z)': f'({count_x}, {count_y}, {count_z})',
        '|X|+|Y|+|Z| (Objective value)': [best_obj],
        'Number of found solutions': [solutions],
        'Time, s': [time],
        'Solver': ['SciOpt'],
        'HardWare' : [hw_info()],
        'x': [x],
        'y': [y],
        "z": [z]
    })
    df_results = pd.concat([df_results, data], ignore_index=False)
    df_results.to_csv("results.csv", index=False)

# Models M1 and M2

In [9]:
class model_m1:
    def __init__(self, G_XY,G_YZ,G_ZX, U,V,W, gamma=1,
                 bounds_U=(-1,-1), bounds_V=(-1,-1), bounds_W=(-1,1)):
        self.Up=U
        self.Vp=V
        self.U = ['x%d'%i for i in range(1,len(U) + 1)]
        self.V = ['y%d'%i for i in range(1,len(V) + 1)]
        self.W = ['z%d'%i for i in range(1,len(W) + 1)]
        self.Wp=W
        self.G_XY  =  G_XY.copy()
        self.G_YZ  =  G_YZ.copy()
        self.G_ZX  =  G_ZX.copy()
        
        #bounds of parts of triclique
        self.lower_U = max(bounds_U[0],1); 
        self.upper_U = max(bounds_U[1], len(U))
        self.lower_V = max(bounds_V[0],1); 
        self.upper_V = max(bounds_V[1], len(V))
        self.lower_W = max(bounds_W[0],1); 
        self.upper_W = max(bounds_W[1], len(W))
        
        #adjacency matrix U-V
        Gdf_XY = nx.to_pandas_adjacency(G_XY)
        self.U_V = list(set(U).intersection(set(Gdf_XY.index.values)))
        # print('qqq - ',)
        #print('qqq - ',set(Gdf_XY.index.values))
        self.V_U = list(set(V).intersection(set(Gdf_XY.index.values)))
        Gdf_XY = (Gdf_XY.loc[self.U_V])[self.V_U]
        # print(Gdf_XY)
        
        #adjacency matrix V-W
        Gdf_YZ = nx.to_pandas_adjacency(G_YZ)
        self.V_W = list(set(V).intersection(set(Gdf_YZ.index.values)))
        self.W_V = list(set(W).intersection(set(Gdf_YZ.index.values)))
        Gdf_YZ = (Gdf_YZ.loc[self.V_W])[self.W_V]
        #print(Gdf_YZ)
        
        #adjacency matrix W-V
        Gdf_ZX = nx.to_pandas_adjacency(G_ZX)
        self.W_U = list(set(W).intersection(set(Gdf_ZX.index.values)))
        self.U_W = list(set(U).intersection(set(Gdf_ZX.index.values)))
        Gdf_ZX = (Gdf_ZX.loc[self.W_U])[self.U_W]
        #print(Gdf_ZX)
        
    
        # indecies of vertices from U to V and vice versa
        U_v_index = list(range(1, len(self.U_V) + 1))
        V_u_index = list(range(1, len(self.V_U) + 1))
        self.Y_UV_index = list(itertools.product(U_v_index,V_u_index))
        #print('U_v_index - ',U_v_index)
        #print('V_u_index - ',V_u_index)
        #print('self.Y_UV_index - ',self.Y_UV_index)
        
        # indecies of vertices from V to W and vice versa
        V_w_index = list(range(1, len(self.V_W) + 1))
        W_v_index = list(range(1, len(self.W_V)+1))
        self.Y_VW_index = list(itertools.product(V_w_index,W_v_index))
        #print('V_w_index - ',V_w_index)
        #print('W_v_index - ',W_v_index)
        #print('self.Y_VW_index - ',self.Y_VW_index)
        
        
        # indecies of vertices from V to W and vice versa
        W_u_index = list(range(1, len(self.W_U) + 1))
        U_w_index = list(range(1, len(self.U_W) + 1))
        self.Y_WU_index = list(itertools.product(W_u_index,U_w_index))
        #print('W_u_index - ',W_u_index)
        #print('U_w_index - ',U_w_index)
        #print('self.Y_WU_index - ',self.Y_WU_index)
        
        
        #Y's
        self.Yij_uv_index = [(i, j) for i, j in self.Y_UV_index if (Gdf_XY.iloc[i-1])[j-1] == 1.]
        self.Yij_uv = ['y(uv)%d_%d'   % (i, j) for i, j in self.Yij_uv_index]
        self.Yij_vw_index = [(i, j) for i, j in self.Y_VW_index if (Gdf_YZ.iloc[i-1])[j-1] == 1.]
        self.Yij_vw = ['y(vw)%d_%d'  % (i, j) for i, j in self.Yij_vw_index]
        self.Yij_wu_index = [(i, j) for i, j in self.Y_WU_index if (Gdf_ZX.iloc[i-1])[j-1] == 1.]
        self.Yij_wu = ['y(wu)%d_%d' %  (i, j) for i, j in self.Yij_wu_index]
        #print('Yij_uv ',self.Yij_uv )
        #print('Yij_vw ',self.Yij_vw )
        #print('Yij_wu ',self.Yij_wu )
        
        
        
        #Z's
        self.Zk_u_index = list(range(self.lower_U, self.upper_U+1))
        self.Zk_v_index = list(range(self.lower_V, self.upper_V+1))
        self.Zk_w_index = list(range(self.lower_V, self.upper_V+1))
        
        self.Zk_u = ['z(u)%d'%i for i in self.Zk_u_index]
        self.Zk_v = ['z(v)%d'%i for i in self.Zk_v_index]
        self.Zk_w = ['z(w)%d'%i for i in self.Zk_w_index]
        #print('Zk_u---',self.Zk_u)
        #print('Zk_v---',self.Zk_v)
        #print('Zk_w---',self.Zk_w)

        self.cplex = None
        self.gamma = gamma
        self.allvars = self.U + self.V +self.W+ self.Yij_uv +self.Yij_vw+self.Yij_wu+ self.Zk_u + self.Zk_v+self.Zk_w
        #print('ALL VARS - ', self.allvars)

    def create_cplex_model(self, path_to_write=None):
        U_v_index = list(range(1, len(self.U_V) + 1))
        V_u_index = list(range(1, len(self.V_U) + 1))
        
        V_w_index = list(range(1, len(self.V_W) + 1))
        W_v_index = list(range(1, len(self.W_V) + 1))
        
        W_u_index = list(range(1, len(self.W_U) + 1))
        U_w_index = list(range(1, len(self.U_W) + 1))

        self.cplex = cplex.Cplex()
        #objective function coeff
        obj = [1.] * (len(self.U) + len(self.V) + len(self.W)) + \
              [0.] * (len(self.Yij_uv)+len(self.Yij_vw) +len(self.Yij_wu)+ len(self.Zk_u) + len(self.Zk_v)+len(self.Zk_w))
            
        #upper bounds coeff of vars
        ub = [1]*(len(self.U)+len(self.V)+len(self.W)) + [cplex.infinity]*len(self.Yij_uv) + \
             [cplex.infinity]*len(self.Yij_vw)+[cplex.infinity]*len(self.Yij_wu)+\
             [1]*(len(self.Zk_u) + len(self.Zk_v)+ len(self.Zk_w))
            
        #lower bound coeff of vars
        lb = [0]*len(self.allvars)
        types = [self.cplex.variables.type.binary]*(len(self.U) + len(self.V)+ len(self.W)) + \
                [self.cplex.variables.type.continuous]*(len(self.Yij_uv)+len(self.Yij_vw)+len(self.Yij_wu)) + \
                [self.cplex.variables.type.binary]*(len(self.Zk_u)+len(self.Zk_v)+len(self.Zk_w))
        
        #maximazing option
        self.cplex.objective.set_sense(self.cplex.objective.sense.maximize)
        self.cplex.variables.add(obj=obj, lb=lb, ub=ub, 
                                 types=types, names=self.allvars)
        
        
        #quadratic costraint
        qct_list_uv = np.array(list(itertools.product(self.Zk_u, self.Zk_v)))
        qct_list_vw = np.array(list(itertools.product(self.Zk_v, self.Zk_w)))
        qct_list_wu = np.array(list(itertools.product(self.Zk_w, self.Zk_u)))
        #print('qct_list',qct_list)
        #print('HMMMM')
        #linear part
        l = cplex.SparsePair(ind = (self.Yij_uv + self.Yij_vw+self.Yij_wu), val = ([1]*len(self.Yij_uv) +\
                             [1]*len(self.Yij_vw)+[1]*len(self.Yij_wu)))
        #print('sparse pair Yi - ',l)
        #quad part
        q = cplex.SparseTriple(ind1=[z1 for z1,z2 in qct_list_uv] + [z1 for z1,z2 in qct_list_vw]+[z1 for z1,z2 in qct_list_wu], 
                               ind2=[z2 for z1,z2 in qct_list_uv]+ [z2 for z1,z2 in qct_list_vw]+[z2 for z1,z2 in qct_list_wu],
                               val=[-1*self.gamma*i*j for i,j in 
                                    itertools.product(self.Zk_u_index,self.Zk_v_index)] +\
                              [-1*self.gamma*i*j for i,j in 
                                    itertools.product(self.Zk_v_index,self.Zk_w_index)]+\
                              [-1*self.gamma*i*j for i,j in 
                                    itertools.product(self.Zk_w_index,self.Zk_u_index)])
        #print('wuad - ',q)
        self.cplex.quadratic_constraints.add(
            lin_expr=l, quad_expr=q, sense="G", rhs=0.0, name='c1')
        
        for num, (i, j) in enumerate(self.Yij_uv_index):
            self.cplex.linear_constraints.add(
                [cplex.SparsePair(ind=['y(uv)%d_%d' % (i, j), 'x%d' % i], val=[1., -1.]),
                 cplex.SparsePair(ind=['y(uv)%d_%d' % (i, j), 'y%d' % j], val=[1., -1.])],
                senses=['L', 'L'], rhs=[0., 0.],
                names=['c2_uv_%d_%d' % (i, j), 'c3_uv_%d_%d' % (i, j)])
            
        for num, (i, j) in enumerate(self.Yij_vw_index):
            self.cplex.linear_constraints.add(
                [cplex.SparsePair(ind=['y(vw)%d_%d' % (i, j), 'y%d' % i], val=[1., -1.]),
                 cplex.SparsePair(ind=['y(vw)%d_%d' % (i, j), 'z%d' % j], val=[1., -1.])],
                senses=['L', 'L'], rhs=[0., 0.],
                names=['c2_vw_%d_%d' % (i, j), 'c3_vw_%d_%d' % (i, j)])
            
            
        for num, (i, j) in enumerate(self.Yij_wu_index):
            self.cplex.linear_constraints.add(
                [cplex.SparsePair(ind=['y(wu)%d_%d' % (i, j), 'z%d' % i], val=[1., -1.]),
                 cplex.SparsePair(ind=['y(wu)%d_%d' % (i, j), 'x%d' % j], val=[1., -1.])],
                senses=['L', 'L'], rhs=[0., 0.],
                names=['c2_wu_%d_%d' % (i, j), 'c3_wu_%d_%d' % (i, j)])
            
        self.cplex.linear_constraints.add(
            [cplex.SparsePair(ind=self.U+self.Zk_u, 
                              val=[1.]*len(self.U) + [-i for i in self.Zk_u_index]),
             cplex.SparsePair(ind=self.V+self.Zk_v, 
                              val=[1.]*len(self.V) + [-i for i in self.Zk_v_index]),
            cplex.SparsePair(ind=self.W+self.Zk_w, 
                              val=[1.]*len(self.W) + [-i for i in self.Zk_w_index])],
            senses=['E', 'E','E'], rhs=[0., 0.,0.], names=['c6_1', 'c6_2', 'c6_3'])

        self.cplex.linear_constraints.add(
            [cplex.SparsePair(ind=self.Zk_u, val=[1.] * len(self.Zk_u)),
             cplex.SparsePair(ind=self.Zk_v, val=[1.] * len(self.Zk_v)),
             cplex.SparsePair(ind=self.Zk_w, val=[1.] * len(self.Zk_w))],
            senses=['E', 'E', 'E'], rhs=[1., 1.,1.], names=['c7_1', 'c7_2', 'c7_3'])

        if not path_to_write is None:
            self.cplex.write(path_to_write)
            
        

    def optimize(self, **kwargs):
        
        """
        Create optimizer and find optimal solutions
        :param gamma: level for quay-clique
        :param kwargs: dict: cplex.mip.parameters: pool_intensity, pool_absgap, limits_populate
        :return: pool of solutions (dict={name: solution_vactor})
        """

        if self.cplex is None:
            self.create_cplex_model()
            self.cplex.solve()

            sol = self.cplex.solution
            print('OPTIMAL SOLUTION - ',sol.get_objective_value())
            #print('SOOOOL - ',sol.get_values())

 

class model_m2:
    def __init__(self, G_XY,G_YZ,G_ZX, U,V,W, gamma=1.0,
                 bounds_U=(-1,-1), bounds_V=(-1,-1), bounds_W=(-1,-1), bounds_X=(-1,-1), bounds_Y=(-1,-1), bounds_Z=(-1,-1)):
        self.Up=U
        self.Vp=V
        self.Wp=W
        
        self.U = ['x%d'%i for i in range(1,len(U) + 1)]
        #print(self.U)
        self.V = ['y%d'%i for i in range(1,len(V) + 1)]
        #print(self.V)
        self.W = ['z%d'%i for i in range(1,len(W) + 1)]
        #print(self.W)

        self.G_XY  =  G_XY.copy()
        self.G_YZ  =  G_YZ.copy()
        self.G_ZX  =  G_ZX.copy()
        
        #bounds of parts of triclique
        self.lower_U = max(bounds_U[0],1); 
        self.upper_U = max(bounds_U[1], len(U))
        self.lower_V = max(bounds_V[0],1); 
        self.upper_V = max(bounds_V[1], len(V))
        self.lower_W = max(bounds_W[0],1); 
        self.upper_W = max(bounds_W[1], len(W))
        
        #adjacency matrix U-V
        #Gdf_XY = nx.to_pandas_dataframe(G_XY)
        Gdf_XY = nx.to_pandas_adjacency(G_XY)
        self.U_V = list(set(U).intersection(set(Gdf_XY.index.values)))
        self.V_U = list(set(V).intersection(set(Gdf_XY.index.values)))
        #print(self.U_V)
        #print(self.V_U)
        Gdf_XY = (Gdf_XY.loc[self.U_V])[self.V_U]
        #print(Gdf_XY)
        
        #adjacency matrix V-W
        Gdf_YZ = nx.to_pandas_adjacency(G_YZ)
        self.V_W = list(set(V).intersection(set(Gdf_YZ.index.values)))
        self.W_V = list(set(W).intersection(set(Gdf_YZ.index.values)))
        Gdf_YZ = (Gdf_YZ.loc[self.V_W])[self.W_V]
        #print(Gdf_YZ)
        
        #adjacency matrix W-V
        Gdf_ZX = nx.to_pandas_adjacency(G_ZX)
        self.W_U = list(set(W).intersection(set(Gdf_ZX.index.values)))
        self.U_W = list(set(U).intersection(set(Gdf_ZX.index.values)))
        Gdf_ZX = (Gdf_ZX.loc[self.W_U])[self.U_W]
        #print(Gdf_ZX)
        
    
        # indecies of vertices from U to V and vice versa
        U_v_index = list(range(1, len(self.U_V) + 1))
        V_u_index = list(range(1, len(self.V_U) + 1))
        self.Y_UV_index = [(i, j) for i, j in list(itertools.product(U_v_index,V_u_index)) if (Gdf_XY.iloc[i-1])[j-1] == 1.]
        #print('U_v_index - ',U_v_index)
        #print('V_u_index - ',V_u_index)
        #print('self.Y_UV_index - ',self.Y_UV_index)
        
        # indecies of vertices from V to W and vice versa
        V_w_index = list(range(1, len(self.V_W) + 1))
        W_v_index = list(range(1, len(self.W_V)+1))
        self.Y_VW_index = [(i, j) for i, j in list(itertools.product(V_w_index,W_v_index)) if (Gdf_YZ.iloc[i-1])[j-1] == 1.]
        #print('V_w_index - ',V_w_index)
        #print('W_v_index - ',W_v_index)
        #print('self.Y_VW_index - ',self.Y_VW_index)
        
        
        # indecies of vertices from V to W and vice versa
        W_u_index = list(range(1, len(self.W_U) + 1))
        U_w_index = list(range(1, len(self.U_W) + 1))
        self.Y_WU_index = [(i, j) for i, j in  list(itertools.product(W_u_index,U_w_index)) if (Gdf_ZX.iloc[i-1])[j-1] == 1.]
        #print('W_u_index - ',W_u_index)
        #print('U_w_index - ',U_w_index)
        #print('self.Y_WU_index - ',self.Y_WU_index)
        
        
        #Y's
        self.Yij_uv = ['y(uv)%d_%d' % (i, j) for i, j in self.Y_UV_index]
        self.Yij_vw = ['y(vw)%d_%d' % (i, j) for i, j in self.Y_VW_index]
        self.Yij_wu = ['y(wu)%d_%d' % (i, j) for i, j in self.Y_WU_index]
        #print('Yij_uv ',self.Yij_uv )
        #print('Yij_vw ',self.Yij_vw )
        #print('Yij_wu ',self.Yij_wu )
        
        #Z's
        self.Zk_u_index = list(range(self.lower_U, self.upper_U+1))
        self.Zk_v_index = list(range(self.lower_V, self.upper_V+1))
        self.Zk_w_index = list(range(self.lower_V, self.upper_V+1))
        
        self.Zk_u = ['z(u)%d'%i for i in self.Zk_u_index]
        self.Zk_v = ['z(v)%d'%i for i in self.Zk_v_index]
        self.Zk_w = ['z(w)%d'%i for i in self.Zk_w_index]
        #print('Zk_u---',self.Zk_u)
        #print('Zk_v---',self.Zk_v)
        #print('Zk_w---',self.Zk_w)


        self.nedjesXY = len(G_XY.edges())
        #print('nedjesXY---',len(G_XY.edges()))
        if bounds_X[0] < 1:
            le = max(int(floor(gamma*self.lower_U*self.lower_V)),1);
            if le >= self.nedjesXY: self.lower_edjesXY = 1
            else: self.lower_edjesXY = le
        else: 
            self.lower_edjesXY = bounds_X[0]
        self.upper_edjesXY = bounds_X[1] if bounds_X[1] > 1 else self.nedjesXY
        
        self.Xa = ['u%d'% i for i in range(self.lower_edjesXY, self.upper_edjesXY+1)]

        self.nedjesYZ = len(G_YZ.edges())
        #print('nedjesYZ---',len(G_YZ.edges()))
        if bounds_Y[0] < 1:
            le = max(int(floor(gamma*self.lower_V*self.lower_W)),1);
            if le >= self.nedjesYZ: self.lower_edjesYZ = 1
            else: self.lower_edjesYZ = le
        else: 
            self.lower_edjesYZ = bounds_Y[0]
        self.upper_edjesYZ = bounds_Y[1] if bounds_Y[1] > 1 else self.nedjesYZ
        
        self.Yb = ['v%d'% i for i in range(self.lower_edjesYZ, self.upper_edjesYZ+1)]


        self.nedjesZX = len(G_ZX.edges())
        #print('nedjesZX---',len(G_ZX.edges()))
        if bounds_Z[0] < 1:
            le = max(int(floor(gamma*self.lower_W*self.lower_U)),1);
            if le >= self.nedjesZX: self.lower_edjesZX = 1
            else: self.lower_edjesZX = le
        else: 
            self.lower_edjesZX = bounds_Y[0]
        self.upper_edjesZX = bounds_Y[1] if bounds_Y[1] > 1 else self.nedjesZX
    
        
        self.Zc = ['w%d'% i for i in range(self.lower_edjesZX, self.upper_edjesZX+1)]
        

        self.cplex = None
        self.gamma = gamma
        self.allvars = self.U + self.V +self.W+ self.Yij_uv +self.Yij_vw+self.Yij_wu+ self.Xa+ self.Yb+ self.Zc+ self.Zk_u + self.Zk_v+self.Zk_w
        #print('ALL VARS - ', self.allvars)

    def create_cplex_model(self, path_to_write=None):
        U_v_index = list(range(1, len(self.U_V) + 1))
        V_u_index = list(range(1, len(self.V_U) + 1))
        
        V_w_index = list(range(1, len(self.V_W) + 1))
        W_v_index = list(range(1, len(self.W_V) + 1))
        
        W_u_index = list(range(1, len(self.W_U) + 1))
        U_w_index = list(range(1, len(self.U_W) + 1))

        self.cplex = cplex.Cplex()
        #objective function coeff
        obj = [0.] * (len(self.U) + len(self.V) + len(self.W)) + \
              [0.] * (len(self.Yij_uv)+len(self.Yij_vw) +len(self.Yij_wu))+\
              [log(k) for k in range(self.lower_edjesXY, self.upper_edjesXY + 1) ] + \
              [log(k) for k in range(self.lower_edjesYZ, self.upper_edjesYZ + 1) ] + \
              [log(k) for k in range(self.lower_edjesZX, self.upper_edjesZX + 1) ] + \
              [-1*log(k) for k in self.Zk_u_index] + [-1*log(k) for k in self.Zk_v_index] + [-1*log(k) for k in self.Zk_w_index]
             
            
        #upper bounds coeff of vars
        ub = [1]*(len(self.U)+len(self.V)+len(self.W)) + [cplex.infinity]*len(self.Yij_uv) + \
             [cplex.infinity]*len(self.Yij_vw)+[cplex.infinity]*len(self.Yij_wu)+\
             [1] * (len(self.Xa)+ len(self.Yb)+ len(self.Zc))+\
             [1]*(len(self.Zk_u) + len(self.Zk_v)+ len(self.Zk_w))
            
        #lower bound coeff of vars
        lb = [0]*len(self.allvars)
        types = [self.cplex.variables.type.binary]*(len(self.U) + len(self.V)+ len(self.W)) + \
                [self.cplex.variables.type.continuous]*(len(self.Yij_uv)+len(self.Yij_vw)+len(self.Yij_wu)) + \
                [self.cplex.variables.type.binary] * len(self.Xa) + \
                [self.cplex.variables.type.binary] * len(self.Yb) +  \
                [self.cplex.variables.type.binary] * len(self.Zc) + \
                [self.cplex.variables.type.binary]*(len(self.Zk_u)+len(self.Zk_v)+len(self.Zk_w))
        
        #maximazing option
        self.cplex.objective.set_sense(self.cplex.objective.sense.maximize)
        self.cplex.variables.add(obj=obj, lb=lb, ub=ub, 
                                 types=types, names=self.allvars)
        


        #linear constraint

        self.cplex.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=self.Yij_uv + self.Xa,
                        val=[1]*len(self.Yij_uv) + 
                        [-k for k in list(range(self.lower_edjesXY,self.upper_edjesXY+1))]), cplex.SparsePair(ind=self.Yij_vw + self.Yb,
                        val=[1]*len(self.Yij_vw) + 
                        [-k for k in list(range(self.lower_edjesYZ,self.upper_edjesYZ+1))]), cplex.SparsePair(ind=self.Yij_wu + self.Zc,
                        val=[1]*len(self.Yij_wu) + 
                        [-k for k in list(range(self.lower_edjesZX,self.upper_edjesZX+1))])],
        senses=['E','E','E'], rhs=[0., 0., 0.], names=['c36_ua','c36_vb','w36_zc'])
         
        #quadratic costraint
        qct_list_uv = np.array(list(itertools.product(self.Zk_u, self.Zk_v)))
        qct_list_vw = np.array(list(itertools.product(self.Zk_v, self.Zk_w)))
        qct_list_wu = np.array(list(itertools.product(self.Zk_w, self.Zk_u)))
        #print('qct_list',qct_list)
        #print('HMMMM')
        #linear part
        l = cplex.SparsePair(ind = (self.Yij_uv + self.Yij_vw+self.Yij_wu), val = ([1]*len(self.Yij_uv) +\
                             [1]*len(self.Yij_vw)+[1]*len(self.Yij_wu)))
        #print('sparse pair Yi - ',l)
        #quad part
        q = cplex.SparseTriple(ind1=[z1 for z1,z2 in qct_list_uv] + [z1 for z1,z2 in qct_list_vw]+[z1 for z1,z2 in qct_list_wu], 
                               ind2=[z2 for z1,z2 in qct_list_uv]+ [z2 for z1,z2 in qct_list_vw]+[z2 for z1,z2 in qct_list_wu],
                               val=[-1*self.gamma*i*j for i,j in 
                                    itertools.product(self.Zk_u_index,self.Zk_v_index)] +\
                              [-1*self.gamma*i*j for i,j in 
                                    itertools.product(self.Zk_v_index,self.Zk_w_index)]+\
                              [-1*self.gamma*i*j for i,j in 
                                    itertools.product(self.Zk_w_index,self.Zk_u_index)])
        #print('wuad - ',q)
        self.cplex.quadratic_constraints.add(
            lin_expr=l, quad_expr=q, sense="G", rhs=0.0, name='c1')
        
        for num, (i, j) in enumerate(self.Y_UV_index):
            self.cplex.linear_constraints.add(
                [cplex.SparsePair(ind=['y(uv)%d_%d' % (i, j), 'x%d' % i], val=[1., -1.]),
                 cplex.SparsePair(ind=['y(uv)%d_%d' % (i, j), 'y%d' % j], val=[1., -1.])],
                senses=['L', 'L'], rhs=[0., 0.],
                names=['c2_uv_%d_%d' % (i, j), 'c3_uv_%d_%d' % (i, j)])
            
        for num, (i, j) in enumerate(self.Y_VW_index):
            self.cplex.linear_constraints.add(
                [cplex.SparsePair(ind=['y(vw)%d_%d' % (i, j), 'y%d' % i], val=[1., -1.]),
                 cplex.SparsePair(ind=['y(vw)%d_%d' % (i, j), 'z%d' % j], val=[1., -1.])],
                senses=['L', 'L'], rhs=[0., 0.],
                names=['c2_vw_%d_%d' % (i, j), 'c3_vw_%d_%d' % (i, j)])
            
            
        for num, (i, j) in enumerate(self.Y_WU_index):
            self.cplex.linear_constraints.add(
                [cplex.SparsePair(ind=['y(wu)%d_%d' % (i, j), 'z%d' % i], val=[1., -1.]),
                 cplex.SparsePair(ind=['y(wu)%d_%d' % (i, j), 'x%d' % j], val=[1., -1.])],
                senses=['L', 'L'], rhs=[0., 0.],
                names=['c2_wu_%d_%d' % (i, j), 'c3_wu_%d_%d' % (i, j)])
            
        self.cplex.linear_constraints.add(
            [cplex.SparsePair(ind=self.U+self.Zk_u, 
                              val=[1.]*len(self.U) + [-i for i in self.Zk_u_index]),
             cplex.SparsePair(ind=self.V+self.Zk_v, 
                              val=[1.]*len(self.V) + [-i for i in self.Zk_v_index]),
            cplex.SparsePair(ind=self.W+self.Zk_w, 
                              val=[1.]*len(self.W) + [-i for i in self.Zk_w_index])],
            senses=['E', 'E','E'], rhs=[0., 0.,0.], names=['c6_1', 'c6_2', 'c6_3'])

        self.cplex.linear_constraints.add(
            [cplex.SparsePair(ind = self.Xa, val = [1.]*len(self.Xa)),
             cplex.SparsePair(ind = self.Yb, val = [1.]*len(self.Yb)),
             cplex.SparsePair(ind = self.Zc, val = [1.]*len(self.Zc))],
            senses=['E', 'E', 'E'], rhs=[1.,1.,1.], names=['c38_ua','c38_vb','c38_wc'])

        self.cplex.linear_constraints.add(
            [cplex.SparsePair(ind=self.Zk_u, val=[1.] * len(self.Zk_u)),
             cplex.SparsePair(ind=self.Zk_v, val=[1.] * len(self.Zk_v)),
             cplex.SparsePair(ind=self.Zk_w, val=[1.] * len(self.Zk_w))],
            senses=['E', 'E', 'E'], rhs=[1., 1.,1.], names=['c7_1', 'c7_2', 'c7_3'])

        if not path_to_write is None:
            self.cplex.write(path_to_write)
            
        

    def optimize(self, **kwargs):
        
        """
        Create optimizer and find optimal solutions
        :param gamma: level for quay-clique
        :param kwargs: dict: cplex.mip.parameters: pool_intensity, pool_absgap, limits_populate
        :return: pool of solutions (dict={name: solution_vactor})
        """

        if self.cplex is None:
            self.create_cplex_model()
        self.cplex.solve()

        sol = self.cplex.solution
        print('OPTIMAL SOLUTION - ',sol.get_objective_value())
        print('SOOOOL - ',sol.get_values())
        self.cplex.parameters.mip.pool.intensity=kwargs.get('pool_intensity',4)
        self.cplex.parameters.mip.pool.absgap=kwargs.get('pool_absgap',0)
        self.cplex.parameters.mip.limits.populate=kwargs.get('limits_populate',1)

        self.cplex.populate_solution_pool()

        sol_dict = {}
        for name in self.cplex.solution.pool.get_names():
            sol_dict[name] = self.cplex.solution.pool.get_values(name, self.allvars)
        print('jjooo - ',self.allvars)
        return self.allvars, sol_dict

# Создаем список с именами классов и ссылками на них
models = [
    {"name": "M1", "class": model_m1},
    {"name": "M2", "class": model_m2}
]

# Выводим содержимое списка
for model in models:
    print(f"Name: {model['name']}, Link: {model['class'].__name__}, Class: {model['class']}")


Name: M1, Link: model_m1, Class: <class '__main__.model_m1'>
Name: M2, Link: model_m2, Class: <class '__main__.model_m2'>


# Config

In [10]:
warnings.filterwarnings("ignore")

# Задаем нижнюю и верхнюю границы gamma
gamma_lower = 0.5
gamma_upper = 1

gamma_step = 0.1

seed_value = 42

timeout = 36000 # Установка маскимального времени на решение // 10 часов = 36000 секунд

# Result

In [11]:
# Подготовка моделей

if not os.path.exists('Model'):
    os.makedirs('Model')

for dataset in datasets:
    for model in models:
        current_gamma = gamma_lower
        while current_gamma <= gamma_upper:
            data = ds_load(dataset)
            model_ = model['class'](*data, gamma=current_gamma)
            cpx_file = f'{model["name"]}_g{current_gamma:.1f}_{dataset["path"]}.lp'
            model_.create_cplex_model(f'Model/{cpx_file}')
            print(f'Dataset: {dataset["name"]} | Model: {model["name"]} | Ɣ: {current_gamma} | CplexFile: {cpx_file}')
            current_gamma += gamma_step

Dataset: Mobile operators | Model: M1 | Ɣ: 0.5 | CplexFile: M1_g0.5_Mobile.lp
Dataset: Mobile operators | Model: M1 | Ɣ: 0.6 | CplexFile: M1_g0.6_Mobile.lp
Dataset: Mobile operators | Model: M1 | Ɣ: 0.7 | CplexFile: M1_g0.7_Mobile.lp
Dataset: Mobile operators | Model: M1 | Ɣ: 0.7999999999999999 | CplexFile: M1_g0.8_Mobile.lp
Dataset: Mobile operators | Model: M1 | Ɣ: 0.8999999999999999 | CplexFile: M1_g0.9_Mobile.lp
Dataset: Mobile operators | Model: M1 | Ɣ: 0.9999999999999999 | CplexFile: M1_g1.0_Mobile.lp
Dataset: Mobile operators | Model: M2 | Ɣ: 0.5 | CplexFile: M2_g0.5_Mobile.lp
Dataset: Mobile operators | Model: M2 | Ɣ: 0.6 | CplexFile: M2_g0.6_Mobile.lp
Dataset: Mobile operators | Model: M2 | Ɣ: 0.7 | CplexFile: M2_g0.7_Mobile.lp
Dataset: Mobile operators | Model: M2 | Ɣ: 0.7999999999999999 | CplexFile: M2_g0.8_Mobile.lp
Dataset: Mobile operators | Model: M2 | Ɣ: 0.8999999999999999 | CplexFile: M2_g0.9_Mobile.lp
Dataset: Mobile operators | Model: M2 | Ɣ: 0.9999999999999999 | Cpl

In [27]:
!ls

Dataset  results.csv		     TEST2.ipynb
Model	 results_mobile_perfume.csv  TEST.ipynb


In [ ]:
for dataset in datasets:
    for model in models:
        current_gamma = gamma_lower
        while current_gamma <= gamma_upper:
            print(f'Dataset: {dataset["name"]} | Model: {model["name"]} | Ɣ: {current_gamma}')
            solve(dataset, model["name"], current_gamma, seed_value)    

            current_gamma += gamma_step

Dataset: Mobile operators | Model: M1 | Ɣ: 0.5
Best objective value: 141.0
(129, 11, 1)
Time: 9.564072
Solutions: 3
Dataset: Mobile operators | Model: M1 | Ɣ: 0.6
Best objective value: 138.0
(129, 8, 1)
Time: 5.06135
Solutions: 2
Dataset: Mobile operators | Model: M1 | Ɣ: 0.7
Best objective value: 136.0
(129, 6, 1)
Time: 8.917704
Solutions: 2
Dataset: Mobile operators | Model: M1 | Ɣ: 0.7999999999999999
Best objective value: 122.0
(116, 5, 1)
Time: 6.132943
Solutions: 4
Dataset: Mobile operators | Model: M1 | Ɣ: 0.8999999999999999
Best objective value: 25.0
(18, 6, 1)
Time: 8378.814587
Solutions: 4
Dataset: Mobile operators | Model: M1 | Ɣ: 0.9999999999999999
Best objective value: 16.0
(14, 1, 1)
Time: 859.188942
Solutions: 1
Dataset: Mobile operators | Model: M2 | Ɣ: 0.5
Best objective value: 171.0
(129, 20, 22)
Time: 75.784755
Solutions: 5
Dataset: Mobile operators | Model: M2 | Ɣ: 0.6
Best objective value: 161.0
(129, 20, 12)
Time: 34.01233
Solutions: 5
Dataset: Mobile operators | M

In [18]:
# Test
# solve(datasets[0], 'M1', gamma_lower, seed_value)    

In [7]:
df_results = pd.read_csv("results.csv")
df_results

,Dataset,Model,Gamma,"First solution (X,Y,Z)",|X|+|Y|+|Z| (Objective value),Number of found solutions,"Time, s",Solver,HardWare,x,y,z
0,Mobile operators,M1,0.5,"(129, 11, 1)",141.0,3,9.564072,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y3', 1.0), ('y4', 1.0), ('y7',...","[('z8', 1.0)]"
1,Mobile operators,M1,0.6,"(129, 8, 1)",138.0,2,5.061350,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y7', 1.0), ('y8', 1.0), ('y9',...","[('z7', 1.0)]"
2,Mobile operators,M1,0.7,"(129, 6, 1)",136.0,2,8.917704,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y7', 1.0), ('y8', 1.0), ('y12'...","[('z8', 1.0)]"
3,Mobile operators,M1,0.8,"(116, 5, 1)",122.0,4,6.132943,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y7', 1.0), ('y8', 1.0), ('y12'...","[('z8', 1.0)]"
4,Mobile operators,M1,0.9,"(18, 6, 1)",25.0,4,8378.814587,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x6',...","[('y1', 1.0), ('y7', 1.0), ('y8', 1.0), ('y12'...","[('z8', 1.0)]"
5,Mobile operators,M1,1.0,"(14, 1, 1)",16.0,1,859.188942,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y8', 1.0)]","[('z14', 1.0)]"
6,Mobile operators,M2,0.5,"(129, 20, 22)",171.0,5,75.784755,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y2', 1.0), ('y3', 1.0), ('y4',...","[('z1', 1.0), ('z2', 1.0), ('z3', 1.0), ('z4',..."
7,Mobile operators,M2,0.6,"(129, 20, 12)",161.0,5,34.012330,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y2', 1.0), ('y3', 1.0), ('y4',...","[('z1', 1.0), ('z2', 1.0), ('z3', 1.0), ('z4',..."
8,Mobile operators,M2,0.7,"(129, 20, 6)",155.0,3,26.148087,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y2', 1.0), ('y3', 1.0), ('y4',...","[('z1', 1.0), ('z2', 1.0), ('z3', 1.0), ('z4',..."
9,Mobile operators,M2,0.8,"(129, 21, 1)",151.0,3,23.049612,SciOpt,Linux | 24 x 12th Gen Intel(R) Core(TM) i9-129...,"[('x1', 1.0), ('x2', 1.0), ('x3', 1.0), ('x4',...","[('y1', 1.0), ('y2', 1.0), ('y3', 1.0), ('y4',...","[('z1', 1.0)]"
